In [1]:
import gzip
import xml.etree.ElementTree as ET
import defusedxml.ElementTree as ET
import pandas as pd
from tqdm.notebook import tqdm
import os
from collections import OrderedDict
import collections
import glob

path_1 = '../Pubmed_original'
path_2 = '../folder_01'
os.chdir(path_1)
print(os.getcwd())
os.chdir(path_2)
print(os.getcwd())
os.chdir(path_1)
print(os.getcwd())

In [2]:
#現在のディレクトリの下の階層に「Pubmed_original」フォルダを配置する
#あらかじめダウンロードしておいたPubmed文献データ(xml.gz)を格納しておく

In [5]:
path_0 = "./Pubmed_original"
path_1 = "../Pubmed_original"
path_2 = '../folder_01'

In [4]:
#あらかじめダウンロードしておいたPubmed文献データ（xml.gz)をリスト化
os.chdir(path_0)
print(os.getcwd())
xml_list = sorted(glob.glob('*.xml.gz'))
print(len(xml_list))

/home/admin_user/Pubmed_original
1062


In [6]:
#要素を取り出す
def getTextFromNode(root, path, fill='', mode=0, attrib='attribute'):
    if (root.find(path) == None):
        return fill
    else:
        if mode == 0:
            return root.find(path).text
        if mode == 1:
            return root.find(path).get(attrib)
#区切る
SEP = '|'

In [7]:
#Qualifier

#Name抽出
def getTextFromNode_quali(root, path, fill='', mode=0, attrib='attribute'):
    name_list = []
    if (root.find(path) == None):
        return fill
    else:
        
        if mode == 0:
            for i in root.findall('QualifierName'):                
                name_list.append(i.text)
            return '@@@'.join(name_list)
        if mode == 1:           
            return root.find(path).get(attrib)



In [8]:
#Qualifier

#UI抽出
def getTextFromNode_quali_UI(root, path, fill='', mode=0, attrib='attribute'):
    name_list = []
    if (root.find(path) == None):
        return fill
    else:
        
        if mode == 1:
            for i in root.findall('QualifierName'):                
                name_list.append(i.get('UI'))
            return '@@@'.join(name_list)


In [9]:
#Qualifier

#MajorTopicYN抽出
def getTextFromNode_quali_YN(root, path, fill='', mode=0, attrib='attribute'):
    name_list = []
    if (root.find(path) == None):
        return fill
    else:
        
        if mode == 1:
            for i in root.findall('QualifierName'):                
                name_list.append(i.get('MajorTopicYN'))
            return '@@@'.join(name_list)



In [10]:
#１個のDescriptorにQualifierが複数ついている ついていないこともある
#Descriptor：Qualifier＝１：１に分割してそれぞれターム名・ID・MajorTopicを取り出す
#PMIDもつける

def one_descriptor_one_qualifier():
    
    pmid_one_desc_one_qual = []
    for num, (pmid, pubdate) in enumerate(zip(df["PMID"], df["PubMedPubDate"])):
        for desc_name, desc_ui, desc_mj, qual_names, qual_uis, qual_mjs in  zip( df["Descriptor"][num], 
                                                                                df["Descriptor_UI"][num],
                                                                                df["Descriptor_MajorTopic"][num],
                                                                                df["Qualifier"][num],
                                                                                df["Qualifier_UI"][num],
                                                                                df["Qualifier_MajorTopic"][num],
                                                                               ):
            for qual_name, qual_ui, qual_mj in zip(qual_names.split('@@@'), qual_uis.split('@@@'), qual_mjs.split('@@@') ):
                pmid_one_desc_one_qual.append([pmid, pubdate, desc_name, desc_ui, desc_mj, qual_name, qual_ui, qual_mj])
    
    tmp_df = pd.DataFrame(pmid_one_desc_one_qual, 
                          columns=["PMID", "PubDate","Descriptor", "Descriptor_UI", "Descriptor_MajorTopic", 
                                                      "Qualifier", "Qualifier_UI","Qualifier_MajorTopic"])
    
    #to csv
    csv_name =  "one_desc_one_qual_" + file[9:-6]+"csv"
    tmp_df.to_csv(csv_name, index=False)
    #print(csv_name)

In [11]:

for num, file in tqdm(enumerate(xml_list[:])):
    #print(file)
    
    input = gzip.open(file, 'r')
    tree = ET.parse(input)
    root = tree.getroot()
    
    articleDics=[]
    for article in tree.iter('PubmedArticle'):
        articleDic = {
            'PMID'                    : getTextFromNode(article, 'MedlineCitation/PMID', ''),
            'PubMedPubDate_Y'         : getTextFromNode(article, 'PubmedData/History/PubMedPubDate[@PubStatus="pubmed"]/Year', ''),
            'PubMedPubDate_M'         : getTextFromNode(article, 'PubmedData/History/PubMedPubDate[@PubStatus="pubmed"]/Month', ''),
            'PubMedPubDate_D'         : getTextFromNode(article, 'PubmedData/History/PubMedPubDate[@PubStatus="pubmed"]/Day', ''),
            'Descriptor'              : SEP.join([getTextFromNode(mesh, 'DescriptorName') for mesh in article.findall('MedlineCitation/MeshHeadingList/')]),
            'Descriptor_UI'           : SEP.join([getTextFromNode(mesh, 'DescriptorName', '', 1, 'UI') for mesh in article.findall('MedlineCitation/MeshHeadingList/')]),
            'Descriptor_MajorTopic'   : SEP.join([getTextFromNode(mesh, 'DescriptorName', '', 1, 'MajorTopicYN') for mesh in article.findall('MedlineCitation/MeshHeadingList/')]),   
            'Qualifier'               : SEP.join([getTextFromNode_quali(mesh, 'QualifierName') for mesh in article.findall('MedlineCitation/MeshHeadingList/')]),
            'Qualifier_UI'            : SEP.join([getTextFromNode_quali_UI(mesh, 'QualifierName', '', 1, 'UI') for mesh in article.findall('MedlineCitation/MeshHeadingList/')]),
            'Qualifier_MajorTopic'    : SEP.join([getTextFromNode_quali_YN(mesh, 'QualifierName', '', 1, 'MajorTopicYN') for mesh in article.findall('MedlineCitation/MeshHeadingList/')]),    
        }
        articleDics.append(OrderedDict(articleDic))
    #データフレームにする
    df = pd.DataFrame(articleDics)
        
    df["PubMedPubDate"] = pd.to_datetime( df["PubMedPubDate_Y"].astype('str')+'-'+df["PubMedPubDate_M"].astype('str').str.zfill(2)+'-'+df["PubMedPubDate_D"].astype('str').str.zfill(2))
    del df["PubMedPubDate_Y"]
    del df["PubMedPubDate_M"]
    del df["PubMedPubDate_D"]
    
    #区切り文字'|'で要素を分割し文字列リストにする        
    df["Descriptor"] = df["Descriptor"].str.split('|')
    df["Descriptor_UI"] = df["Descriptor_UI"].str.split('|')
    df["Descriptor_MajorTopic"] = df["Descriptor_MajorTopic"].str.split('|')
    df["Qualifier"] = df["Qualifier"].str.split('|')
    df["Qualifier_UI"] = df["Qualifier_UI"].str.split('|')
    df["Qualifier_MajorTopic"] = df["Qualifier_MajorTopic"].str.split('|')
       

    os.chdir(path_2)
    one_descriptor_one_qualifier()
    
    os.chdir(path_1)

0it [00:00, ?it/s]

In [12]:
#生成件数
os.chdir(path_2)
csv_list = sorted(glob.glob('*.csv'))
print(len(csv_list), csv_list[:1])

1 ['one_desc_one_qual_0001.csv']
